In [39]:
import pandas as pd

git_log = pd.read_csv(
    "../../buschmais-spring-petclinic/git.log",
    sep="\u0012",
    names=['raw'])
git_log.head()

,raw
0,commit cf842224b8e250f561a0e65a740d2817a3cae61f
1,Author: Dirk Mahler <dirk.mahler@buschmais.com>
2,Date: Mon Mar 26 13:22:32 2018 +0200
3,upgraded to Git plugin 1.3.3
4,commit c3a6d6c1501033311a1021d4fc142db50c0183de


In [46]:
# TODO hier fehlt noch der Marker, was zusammengehört (aka commit id)
git_log['type'] = git_log.raw.\
                    str.split(": ", n=1).\
                    str[0].str.title()
git_log.type = git_log.type.fillna("message")
git_log.head()

,raw,type
0,commit cf842224b8e250f561a0e65a740d2817a3cae61f,Commit Cf842224B8E250F561A0E65A740D2817A3Cae61F
1,Author: Dirk Mahler <dirk.mahler@buschmais.com>,Author
2,Date: Mon Mar 26 13:22:32 2018 +0200,Date
3,upgraded to Git plugin 1.3.3,Upgraded To Git Plugin 1.3.3
4,commit c3a6d6c1501033311a1021d4fc142db50c0183de,Commit C3A6D6C1501033311A1021D4Fc142Db50C0183De


In [44]:
pd.melt(git_log, col_level=0)

,variable,value
0,raw,commit cf842224b8e250f561a0e65a740d2817a3cae61f
1,raw,Author: Dirk Mahler <dirk.mahler@buschmais.com>
2,raw,Date: Mon Mar 26 13:22:32 2018 +0200
3,raw,upgraded to Git plugin 1.3.3
4,raw,commit c3a6d6c1501033311a1021d4fc142db50c0183de
5,raw,Merge: 8fb11ea 887eafe
6,raw,Author: Dirk Mahler <dirk.mahler@buschmais.com>
7,raw,Date: Sun Oct 29 12:26:56 2017 +0100
8,raw,Merge pull request #5 from feststelltaste/...
9,raw,Update demo output to new version


In [26]:
git_log.pivot_table(
    index=git_log.index,
    columns=["type"],
    aggfunc='first')

raw  \
type                                                      
0                                                  None   
1                                                  None   
2                                                  None   
3                          upgraded to Git plugin 1.3.3   
4                                                  None   
5                                                  None   
6                                                  None   
7                                                  None   
8         Merge pull request #5 from feststelltaste/...   
9                     Update demo output to new version   
10                                                 None   
11                                                 None   
12                                                 None   
13                                     Fix another typo   
14                                                 None   
15                                                 None   
16                                                 None   
17                                       fix minor typo   
18                                                 None   
19                                                 None   
20                                                 None   
21                                update to new version   
22                                                 None   
23                                                 None   
24                                                 None   
25                                                 None   
26                       Merge branch 'asciidoc-report'   
27                                         # Conflicts:   
28                                      #       pom.xml   
29                                                 None   
...                                                 ...   
2553                                               None   
2554                         upgrade to spring 3.0.0.m3   
2555                                               None   
2556                                               None   
2557                                               None   
2558                                    petclinic tiles   
2559                                               None   
2560                                               None   
2561                                               None   
2562                                            logging   
2563                                               None   
2564                                               None   
2565                                               None   
2566                                 will add css later   
2567                                               None   
2568                                               None   
2569                                               None   
2570                                             styles   
2571                                               None   
2572                                               None   
2573                                               None   
2574              app deploys and runs - initial commit   
2575                                               None   
2576                                               None   
2577                                               None   
2578                   petclinic 3 initial commit - wip   
2579                                               None   
2580                                               None   
2581                                               None   
2582                                     Initial commit   

                                                         \
type                                             Author   
0                                                  None   
1       Author: Dirk Mahler <dirk.mahler@buschmais.com>   
2                                                  None   
3                                      